In [2]:
import os

# Numbers
import pandas as pd
import numpy as np

# Graphics
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from termcolor import (
    colored,
    cprint,
)

# Modeling
from sklearn.preprocessing import (
    MinMaxScaler,
    StandardScaler,
    OneHotEncoder,
    LabelBinarizer,
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
)
import keras

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

import tensorflow as tf

# from keras.layers import (
#     Dense,
#     Dropout,
#     Flatten,
#     Activation,
#     BatchNormalization,
# )
# from keras.callbacks import EarlyStopping

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Activation,
    Dense, Dropout, Flatten, BatchNormalization,
    Conv2D, MaxPooling2D,
)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image


###### Voor Tensorflow-GPU ########
import re

#K.set_image_dim_ordering('tf')

# Make sure that we have pandas.DataFrames at each step in scikit-learn
from sklearn import set_config
set_config(transform_output="pandas")

# Multiprocessing options (joblib/loky)
os.environ['OMP_NUM_THREADS']='8'

# limit some output when printing pandas data
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)
pd.set_option('mode.copy_on_write', True)
pd.set_option('future.no_silent_downcasting', True)

# Some defaults for matplotlib
plt.rcParams['image.cmap'] = 'gray'
# plt.tight_layout()
# plt.rcParams.update({
#     "axes.titlesize":"10",
#     "xtick.labelsize":"8",
#     "ytick.labelsize":"8",
#     "font.size": "8",
#     # "figure.figsize": [6,3],
# })

###### Voor Tensorflow-GPU ########

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# retrain the model with class_weight
from sklearn.utils import class_weight
 
number_of_hidden_layers = 1
number_of_neurons = 256
dropoutrate = 0.3
epochs = 25
batch_size = 64
 
# sequential model
model = Sequential()
 
# input layer
model.add(Dense(number_of_neurons, input_dim=X_train.shape[1], kernel_initializer='RandomUniform', activation='relu'))
model.add(Dropout(dropoutrate))
model.add(BatchNormalization())
 
# hidden layers
for _ in range(number_of_hidden_layers):
    number_of_neurons = int(number_of_neurons / 2)
    model.add(Dense(number_of_neurons, kernel_initializer='RandomUniform', activation='relu'))
    model.add(Dropout(dropoutrate))
    model.add(BatchNormalization())
 
# output layer
model.add(Dense(2, kernel_initializer='RandomUniform', activation='softmax'))
 
model.compile(loss='binary_crossentropy', optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
 
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=1, class_weight=class_weights)
 
plot_history(history)
 
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
 
print('\n')
print(classification_report(y_test, y_pred))
 
cf = confusion_matrix(y_test, y_pred)
print(cf)
print(accuracy_score(y_test, y_pred) * 100)

NameError: name 'X_train' is not defined